In [0]:
# At the very top of 04_model_training.py
# Install Prophet and dependencies scoped to this cluster/session
%pip install prophet pandas

# After install completes, Databricks will suggest a restart; click "Restart" or run:
dbutils.library.restartPython()

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:

# 04_model_training.py

# Install Prophet (Databricks supports %pip)
# %pip install prophet pandas

import pandas as pd
from pyspark.sql.functions import col
from prophet import Prophet

CATALOG = "workspace"
SCHEMA  = "default"
gold_tbl     = f"{CATALOG}.{SCHEMA}.stocks_gold_features"
forecast_tbl = f"{CATALOG}.{SCHEMA}.stocks_forecast"

symbol = "AAPL"  # choose one symbol for now

# Load data for the chosen symbol
df = (spark.table(gold_tbl)
          .filter(col("symbol") == symbol)
          .select("date", "close")
          .orderBy("date"))

pdf = df.toPandas().dropna()
pdf = pdf.rename(columns={"date": "ds", "close": "y"})

# Train Prophet model
m = Prophet(daily_seasonality=True)
m.fit(pdf)

# Forecast next 7 days
future = m.make_future_dataframe(periods=7, freq="D")
forecast = m.predict(future)

# Prepare predictions
preds = forecast[["ds", "yhat", "yhat_lower", "yhat_upper"]].copy()
preds["symbol"] = symbol

# Write to managed Delta table
spark.createDataFrame(preds).write.mode("overwrite").saveAsTable(forecast_tbl)

display(spark.table(forecast_tbl))


07:10:16 - cmdstanpy - INFO - Chain [1] start processing
07:10:16 - cmdstanpy - INFO - Chain [1] done processing


ds,yhat,yhat_lower,yhat_upper,symbol
2014-01-02T00:00:00.000Z,74.88599953750439,69.84464630375359,80.26022085512639,AAPL
2014-01-03T00:00:00.000Z,74.66122203711517,69.58276784519622,79.87639747013144,AAPL
2014-01-06T00:00:00.000Z,74.22171851901189,69.16545699716418,79.8505940201166,AAPL
2014-01-07T00:00:00.000Z,74.02679048295695,68.42718151815937,79.47661035556422,AAPL
2014-01-08T00:00:00.000Z,73.85109650404594,68.7315401391805,79.09542367043882,AAPL
2014-01-09T00:00:00.000Z,73.49422006773415,68.42663710080784,78.46547267182734,AAPL
2014-01-10T00:00:00.000Z,73.27663779637729,68.19094377423882,78.31702768000441,AAPL
2014-01-13T00:00:00.000Z,72.89185323387903,67.94621392702619,77.809468710158,AAPL
2014-01-14T00:00:00.000Z,72.7317812774721,67.45218410938715,77.97014688227739,AAPL
2014-01-15T00:00:00.000Z,72.60187030117368,67.4760695307714,77.87131299448562,AAPL
